# Generate augmentations and split files
#### 12 augmentations
#### 1 original
#### Affine transform: Both original image and depth map must be augmented

In [1]:
import os
import os.path as osp
import cv2
import shutil
import random
import pandas as pd
from tqdm.notebook import tqdm
from augmentations import augmentation_choices, transform_image

In [2]:
# windows will accept '/' as separator
split_file = 'kitti_eigen_test.txt'
input_prefix_path = "E:/Thesis_Datasets/kitti_full/"
input_depth_folder = 'val'
output_prefix_path = "E:/Thesis_Datasets/Augmented_Images/"
output_image_folder = "Data/"
output_depth_folder = "Depth/"

In [3]:
valid_image_count = 0
with open(split_file) as f:
    for line in tqdm(f, desc="Counting input files...", total=652):
        img_name = line.strip().split(" ")[0]
        img_name = osp.join(input_prefix_path, img_name)
        depth_file = line.strip().split(" ")[1]
        depth_map = osp.join(input_prefix_path, input_depth_folder, depth_file)

        if not osp.isfile(depth_map):
            depth_map = osp.join(input_prefix_path, 'train', depth_file)
        if osp.isfile(img_name) and osp.isfile(depth_map):
            valid_image_count += 1
        
print("Total valid images: ", valid_image_count)

Counting input files...:   0%|          | 0/652 [00:00<?, ?it/s]

Total valid images:  652


In [4]:
for aug in augmentation_choices:
    print(aug)

None
gaussian
localvar
poisson
salt
pepper
s&p
speckle
affine
rain
shadow
flare
snow


In [ ]:
# augmentation_choices = ['gaussian', 'affine']

In [ ]:
count = 0
with open(split_file) as f:
    for line in f:
        count += 1
        img_name = line.strip().split(" ")[0]
        img_name = osp.join(input_prefix_path, img_name)
        depth_file = line.strip().split(" ")[1]
        depth_map = osp.join(input_prefix_path, input_depth_folder, depth_file)
#         scale=(1.1, 1.1)
#         rotation=random.uniform(0.00, 0.0)
    #         shear=random.uniform(0.0, 0.0)

            scale=None
            rotation=None
            shear=None

            original_depth = cv2.imread(depth_map, cv2.IMREAD_UNCHANGED)
        augmented_depth = transform_image(aug, original_depth, scale, rotation, shear)
        
        print(f"Dimensions of depth map: \t{original_depth.shape}")
        print(f"Max of depth map: \t{original_depth.max()}")
        print(f"Type of max of depth map: {type(original_depth.max())}")
        print(f"Dimensions of transformed depth: {augmented_depth.shape}")
        print(f"Max of transformed depth map: {augmented_depth.max()}")
        

        cv2.imwrite(osp.join("./" + img_name.split('/')[-1]), augmented_depth)
        break

In [ ]:
filepath = osp.normpath(osp.join(input_prefix_path, input_depth_folder, depth_file)).replace('\\', '/')

# print(range(len(filepath.split('/'))-1))
print(filepath)

for i in range(len(filepath.split('/'))-1):
    current_dir = osp.join('', *(filepath.split('/')[:i+1]))
    print(f"current directory: {current_dir}")
    print(f"exists: {osp.isdir(current_dir)}")
    

In [ ]:
count_good_pixels = 0
for i in range(original_depth.shape[0]):
    for j in range(original_depth.shape[1]):
        if(original_depth[i][j] == augmented_depth[i][j]):
            count_good_pixels += 1
print(count)
print(original_depth[200][115:120])
print(augmented_depth[200][115:120])

In [5]:
def make_subdir(path):
    filepath = osp.normpath(path).replace('\\', '/')
                
    for i in range(len(filepath.split('/'))):
        current_dir = osp.join('', *(filepath.split('/')[:i+1]))            
     
        if not osp.isdir(current_dir):
            try:
#                 print(f"Making {current_dir}")
                os.mkdir(current_dir)
            except FileNotFoundError:
                print(f"Failed to make {current_dir}. Make sure the parent folder exists.")                

In [6]:
depth_split

NameError: name 'depth_split' is not defined

In [11]:
invalid_image_count = 0
valid_image_count = 0
image_data = []

with open(split_file) as f:
    for line in tqdm(f, desc="Augmenting input files...", total=652):
        img_split = line.strip().split(" ")[0]
        img_filename = osp.join(input_prefix_path, img_split)
        depth_postfix = line.strip().split(" ")[1]
        depth_filename = osp.join(input_prefix_path, input_depth_folder, depth_postfix)
        
        if not osp.isfile(depth_filename):
            depth_filename = osp.join(input_prefix_path, 'train', depth_postfix)

        if osp.isfile(img_filename) and osp.isfile(depth_filename):
            valid_image_count += 1

            # Ensure the folder exists
            make_subdir(osp.join(output_prefix_path, output_depth_folder, osp.join('', *depth_postfix.split('/')[:-1])))

            #   Copy depth map
            shutil.copy(depth_filename,
                        osp.join(output_prefix_path, 
                                 output_depth_folder, 
                                 osp.join('', *depth_postfix.split('/')[:-1]), depth_filename.split('/')[-1]))

            #   For each augmentation type:
            for aug in augmentation_choices:
                if aug == None:
                    aug = "None"
                
                make_subdir(osp.join(output_prefix_path, output_image_folder, aug, *img_split.split('/')[:-1]))

                # For all other augmentation types:          
                if aug != 'affine':
                    img = transform_image(aug, cv2.imread(img_filename))
                    # todo: fix list
                    # Insert [rgb_file, depth_prefix, augmentation_type] into list
                    image_data.append([osp.join(output_image_folder, 
                                                aug,
                                                osp.join('', *img_split.split('/')[:-1]).replace('\\', '/'),
                                                img_filename.split('/')[-1]
                                               ),
                                       osp.join(output_depth_folder,
                                                osp.join('', *depth_postfix.split('/')[:-1]).replace('\\', '/'),
                                                depth_postfix.split('/')[-1]),
                                       aug])
                    
                # if aug = affine, depth map needs to be augmented with the same scale, rotation and shear
                else:
                    make_subdir(osp.join(output_prefix_path, output_depth_folder, aug, *depth_postfix.split('/')[:-1]))

                    scale=(1.1, 1.1)
                    rotation=random.uniform(-0.05, 0.05)
                    shear=random.uniform(-0.1, 0.1)

                    img = transform_image(aug, cv2.imread(img_filename), scale, rotation, shear)
                    
                    # Note: Read as UNCHANGED (ie grayscale), otherwise imwrite will save RGB channels
                    augmented_depth = transform_image(aug, cv2.imread(depth_filename, cv2.IMREAD_UNCHANGED), scale, rotation, shear)

                    # todo: change write location
                    # Save transformed depth map in Depth/affine/xxxxxxx_affine.png
                    cv2.imwrite(osp.join(output_prefix_path, 
                                     output_depth_folder, 
                                     aug,
                                     osp.join('', *depth_postfix.split('/')[:-1]).replace('\\', '/'),
                                     img_filename[img_filename.rfind('/')+1:-4] + "_affine.png"),
                                augmented_depth)
                    
                    # todo: fix list
                    # Insert  [rgb_file, 
                    #          depth_split, 
                    #          augmentation_type] 
                    #   into list
                    image_data.append([osp.join(output_image_folder, 
                                                aug, 
                                                osp.join('', *img_split.split('/')[:-1]).replace('\\', '/'), 
                                                img_filename.split('/')[-1]
                                               ),
                                       osp.join(output_depth_folder, 
                                                aug,
                                                osp.join('', *depth_postfix.split('/')[:-1]).replace('\\', '/'),
                                                depth_filename.split('/')[-1][:-4] + "_affine.png"),
                                       aug]
                                     )
                    
                # todo: change write location
                # Save augmented RGB file with new name in /<augmentation_type>/ folder
                cv2.imwrite(
                    osp.join(output_prefix_path, 
                                     output_image_folder, 
                                     aug, 
                                     osp.join('', *img_split.split('/')[:-1]).replace('\\', '/'),
                                     img_filename.split('/')[-1]),
                    img
                )

        else:
            invalid_image_count += 1


print(f"Total augmented files: {valid_image_count}")
print(f"Total invalid files: {invalid_image_count}")


Augmenting input files...:   0%|          | 0/652 [00:00<?, ?it/s]

Total augmented files: 652
Total invalid files: 45


In [ ]:
augmentation_choices

In [ ]:
osp.join(output_prefix_path, output_depth_folder, aug, *depth_split.split('/')[:-1])
# make_subdir(test_dir)
# cv2.imwrite(test_dir, augmented_depth)

In [ ]:
osp.join('', *img_split.split('/')[:-1]).replace('\\', '/')

In [ ]:
print(img_split)
print(depth_filename)
print(img_split.split('/')[:-1])
print(depth_split)

In [ ]:
sorted(image_data_df['image'])

In [ ]:
osp.join(output_prefix_path, 
                                     output_image_folder, 
                                     aug, 
                                     img_name.split('/')[-1])

In [ ]:
# print(osp.join(output_prefix_path, 
#                                      output_image_folder, 
#                                      aug, 
#                                      img_name.split('/')[-1]))

In [ ]:
# image_data_df['image'][0].replace

In [12]:
# Ensure we are using linux-style '/' instead of windows '\' or '\\'

image_data_df = pd.DataFrame(image_data, columns=['image', 'depth', 'augmentation'])
image_data_df['image'] = image_data_df['image'].apply(lambda x: x.replace('\\', r'/'))
image_data_df['depth'] = image_data_df['depth'].apply(lambda x: x.replace('\\', r'/'))
image_data_df.head(20)
# image_data_df.depth[8]


,image,depth,augmentation
0,Data/None/2011_09_26/2011_09_26_drive_0002_syn...,Depth/2011_09_26_drive_0002_sync/proj_depth/gr...,None
1,Data/gaussian/2011_09_26/2011_09_26_drive_0002...,Depth/2011_09_26_drive_0002_sync/proj_depth/gr...,gaussian
2,Data/localvar/2011_09_26/2011_09_26_drive_0002...,Depth/2011_09_26_drive_0002_sync/proj_depth/gr...,localvar
3,Data/poisson/2011_09_26/2011_09_26_drive_0002_...,Depth/2011_09_26_drive_0002_sync/proj_depth/gr...,poisson
4,Data/salt/2011_09_26/2011_09_26_drive_0002_syn...,Depth/2011_09_26_drive_0002_sync/proj_depth/gr...,salt
5,Data/pepper/2011_09_26/2011_09_26_drive_0002_s...,Depth/2011_09_26_drive_0002_sync/proj_depth/gr...,pepper
6,Data/s&p/2011_09_26/2011_09_26_drive_0002_sync...,Depth/2011_09_26_drive_0002_sync/proj_depth/gr...,s&p
7,Data/speckle/2011_09_26/2011_09_26_drive_0002_...,Depth/2011_09_26_drive_0002_sync/proj_depth/gr...,speckle
8,Data/affine/2011_09_26/2011_09_26_drive_0002_s...,Depth/affine/2011_09_26_drive_0002_sync/proj_d...,affine
9,Data/rain/2011_09_26/2011_09_26_drive_0002_syn...,Depth/2011_09_26_drive_0002_sync/proj_depth/gr...,rain


Create a split_file_<aug>.txt for each augmentation type, and save the full dataframe as a pickle.

In [13]:
for aug in augmentation_choices[1:]:
    subset = image_data_df.loc[image_data_df.augmentation == aug]
    subset[['image', 'depth']].to_csv('E:\Thesis_Datasets\Augmented_Images\split_file_'+aug+'.txt', sep=' ', index=False, header=False)
image_data_df.to_pickle('image_data_df.pkl')

In [14]:
subset = image_data_df.loc[image_data_df.augmentation == 'None']
subset[['image', 'depth']].to_csv('E:\Thesis_Datasets\Augmented_Images\split_file_'+'None.txt', sep=' ', index=False, header=False)

In [ ]:
subset.head()